In [1]:
import pandas as pd
import numpy as np

In [2]:
! pip install arabert
! pip install transformers
! pip install farasapy
! pip install pyarabic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
data=pd.read_csv('/content/drive/MyDrive/dataClean.csv')

In [4]:
data=data.dropna()

In [5]:
map_label = {
    'سياسة': 0,
    'اقتصاد': 1,
    'صحة': 2
}
label_map={
   0:'سياسة',
    1:'اقتصاد',
    2:'صحة'}

In [6]:
data['category'] = data['category'].map(map_label)

In [7]:
from sklearn.model_selection import train_test_split
X=data['cleandata']
Y=data['category']
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,train_size=0.8,random_state=42)

In [8]:
from arabert.preprocess import ArabertPreprocessor
model_name="bert-base-arabert"
arabert_prep = ArabertPreprocessor(model_name=model_name)

[2023-06-07 19:56:41,115 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [9]:
X_train0=X_train.apply(lambda x:arabert_prep.preprocess(x))
X_test0=X_test.apply(lambda x:arabert_prep.preprocess(x))

In [10]:
from arabert.preprocess import ArabertPreprocessor
from sklearn.metrics import (accuracy_score, f1_score,recall_score)
from torch.utils.data import  Dataset
from transformers import (AutoConfig, AutoModelForSequenceClassification,
                        AutoTokenizer, BertTokenizer, Trainer,
                        TrainingArguments)
from transformers.data.processors.utils import InputFeatures

In [11]:
#chose bert model
model_name = 'aubmindlab/bert-base-arabert'
#asafaya/bert-base-arabic
#UBC-NLP/ARBERT
#UBC-NLP/MARBERT
#bert-base-multilingual-uncased
num_labels =3
max_length = 120

In [12]:

class ClassificationDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
        super(ClassificationDataset).__init__()
        self.text = text
        self.target = target
        self.tokenizer_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.max_len = max_len
        self.label_map = label_map
      

    def __len__(self):
        return len(self.text)

    def __getitem__(self,item):
        text = str(self.text[item])
        text = " ".join(text.split())
        
        inputs = self.tokenizer(
          text,
          max_length=self.max_len,
          padding='max_length',
          truncation=True
        )      
        return InputFeatures(**inputs,label= self.target[item])

In [13]:
Y_train=Y_train.astype(int)
Y_test =Y_test.astype(int)

In [14]:
train_dataset = ClassificationDataset(
    X_train0.to_list(),
    Y_train.to_list(),
    model_name,
    max_length,
    map_label
)
test_dataset = ClassificationDataset(
    X_test0.to_list(),
    Y_test.to_list(),
    model_name,
    max_length,
    map_label
)

In [15]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=num_labels)

In [16]:
def compute_metrics(p):
    #p should be of type EvalPrediction
    preds = np.argmax(p.predictions, axis=1)
    assert len(preds) == len(p.label_ids)
    macro_f1 = f1_score(p.label_ids,preds,average='macro')
    macro_recall = recall_score(p.label_ids,preds,average='macro')
    acc = accuracy_score(p.label_ids,preds)
    return {       
      'macro_f1' : macro_f1,
      'accuracy': acc,
      'recall':macro_recall
  }

In [17]:
pip uninstall -y transformers accelerate


Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Successfully uninstalled transformers-4.29.2
Found existing installation: accelerate 0.20.1
Uninstalling accelerate-0.20.1:
  Successfully uninstalled accelerate-0.20.1


In [19]:
pip install transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Using cached accelerate-0.20.1-py3-none-any.whl (227 kB)


In [20]:
training_args = TrainingArguments( 
    output_dir= "./train",    
    adam_epsilon = 1e-8,
    learning_rate = 2e-5,
    fp16 = False, # enable this when using V100 or T4 GPU
    per_device_train_batch_size = 16, # up to 64 on 16GB with max len of 128
    per_device_eval_batch_size = 16,
    gradient_accumulation_steps = 4, # use this to scale batch size without needing more memory
    num_train_epochs= 8,
    warmup_ratio = 0,
    do_eval = True,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    save_total_limit=10,
    load_best_model_at_end = True, # this allows to automatically get the best model at the end based on whatever metric we want
    metric_for_best_model = 'eval_loss',
    greater_is_better = False,
    report_to=[]
  )

In [21]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at aubmindlab/bert-base-arabert were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [22]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Macro F1,Accuracy,Recall
1,No log,0.119472,0.961596,0.961794,0.961667
2,No log,0.122071,0.964485,0.964563,0.964450
3,No log,0.131291,0.964396,0.964563,0.964498
4,No log,0.155044,0.963325,0.963455,0.963413
5,0.104000,0.172922,0.962245,0.962348,0.962253
6,0.104000,0.185336,0.963293,0.963455,0.963374
7,0.104000,0.186630,0.962227,0.962348,0.962228
8,0.104000,0.189279,0.962212,0.962348,0.962279


TrainOutput(global_step=904, training_loss=0.06349556319481503, metrics={'train_runtime': 1555.861, 'train_samples_per_second': 37.14, 'train_steps_per_second': 0.581, 'total_flos': 3563377905467520.0, 'train_loss': 0.06349556319481503, 'epoch': 8.0})

In [23]:
trainer.model.config.label2id = map_label
trainer.model.config.id2label = label_map


In [25]:
#sauvegarde de model
trainer.save_model("./model")
train_dataset.tokenizer.save_pretrained("./model")

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.txt',
 './model/added_tokens.json',
 './model/tokenizer.json')

In [26]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("./model")

In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./model")

In [ ]:
text="القدس المحتلة- أعادت حادثة مقتل 3 جنود إسرائيليين برصاص شرطي مصري بالقرب من معبر العوجة داخل الأراضي الإسرائيلية في صحراء النقب إلى الأذهان العملية المسلحة التي نفذها جندي مصري قبل 33 عاما بالمنطقة الحدودية وأسفرت عن مقتل 5 إسرائيليين، بينهم 3 جنود"
# Prétraitement du texte avec le tokenizer
inputs = tokenizer(text, return_tensors="pt")

# Effectuer la prédiction avec le modèle
outputs = model(**inputs)

# Obtenir les prédictions
predictions = outputs.logits.argmax(dim=1)
predicted_category = map_label[predictions.item()] 
print(predicted_category)